# Generative Adversial Networks(GANs)

They were introduced by Ian Goodfellow in his research paper in 2014. According to Facebook's AI research director Yann LeCun GANs are the most interesting idea in the last 10 years in ML. It consits of two networks a discriminator and a generator. A generator is a network that tries to create fake data identical to real data and discriminator tries to distinguish between fake and real data. To give an example let's assume Genrator creates counterfeit money and it's the job of the Discriminator to distinguish between real money and counterfeit money. Over time the Generator becomes better in creating counterfeit money which is identical to real money. 

In [1]:
import pickle as pkl  #Importing Modules
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os,imageio,itertools,time,pickle
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True,reshape=[]) #one hot true does one hot encoding
tf.reset_default_graph()
batch_size=100
lr=0.0002
epochs=100
display_epoch = 1
logs_path = 'vgan'
smooth = 0.1


## Discriminator
The discriminator takes a 28X28 matrix and tries to predict wether it is fake or real. It contains only one hidden layer of 128 neurons.

In [3]:
def discriminator(x,reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        #layer1
        l1=tf.layers.dense(x, 128, activation=None)
        l1=tf.nn.leaky_relu(l1,alpha=0.01)
        #layer2
        l2=tf.layers.dense(l1, 1, activation=None)
        o=tf.nn.sigmoid(l2)
        return o,l2

## Generator
The generator tries to predict the real data distribution. It takes a random uniform distribution as input and learns to map it to the real data. It also contains one hidden layer of 128 neurons.

In [4]:
def generator(z_dim,reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        #layer1
        l1=tf.layers.dense(z_dim, 128, activation=None)
        l1=tf.nn.leaky_relu(l1,alpha=0.01)
        #layer2
        l2=tf.layers.dense(l1, 784, activation=None)
        l2=tf.nn.tanh(l2)
        return l2


### Visualization
The following few lines of code help in visualization.

In [5]:
root = 'MNIST_DCGAN_results/'
model = 'MNIST_DCGAN_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')

fixed_z_ = np.random.uniform(-1, 1, size=(batch_size, 100))
def show_result(num_epoch, show = False, save = False, path = 'result.png'):

    test_images = sess.run(g_i, {z_dim: fixed_z_,})
    test_images=np.reshape(test_images,[-1,28,28,1])
    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)
    for k in range(size_figure_grid*size_figure_grid):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (28, 28)), cmap='gray')
    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()


In [6]:
with tf.name_scope('inputs') as inputs: #Inputs Of Our Network
        x = tf.placeholder(tf.float32, shape=(None,784), name='x')
        z_dim = tf.placeholder(tf.float32, shape=(None,100), name='z_dim') #z_dim is for our random distribution.

In [7]:
g = generator(z_dim,reuse=False) #Generator takes the input z_dim and returns a 28*28(784) matrix.
g_i = generator(z_dim,reuse=True) #Reuse of variables is True as g_i will be used for visualization.
gs=tf.summary.image("generator_i",tf.reshape(g,shape=(-1,28,28,1)), max_outputs=10) #Tensorboard Image Operation

In [8]:
d_t ,d_tlog= discriminator(x,reuse=False) #Real Data as Input
d_f ,d_flog= discriminator(g,reuse=True) #Fake Data as Input and Reuse of Variables as True

In [9]:
with tf.name_scope('Discriminator_Loss_Fake'): #The Discriminator will set Generator's Fake Data as 0
    dfcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_flog, labels=tf.zeros_like(d_f)))
    dfcosts=tf.summary.scalar("Discriminator_Loss_Fake",dfcost)

with tf.name_scope('Discriminator_Loss_Real'): #The Discriminator will set Real Data input as 1. 1-smooth is used for label smoothning.
    dtcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_tlog, labels= tf.ones_like(d_t)*(1-smooth)))
    dtcosts=tf.summary.scalar("Discriminator_Loss_Real",dtcost)

with tf.name_scope('Discriminator_Loss'): #The Two losses are added
    dcost=dtcost+dfcost
    dcosts=tf.summary.scalar('Discriminator_Loss',dcost)


In [10]:
with tf.name_scope('Generator_Loss'): #The Generator will try to fool the Discriminator 
    gcost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_flog, labels=tf.ones_like(d_f)))
    gcosts=tf.summary.scalar('Generator_Loss',gcost)

In [11]:
T_vars = tf.trainable_variables() #Making list of Trainable Variables
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): #Using Adam Optimizer
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(dcost, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(gcost, var_list=G_vars)
#Tensorboard Operation
d_summary=tf.summary.merge([dcosts])
g_summary=tf.summary.merge([gcosts,gs])

In [13]:
saver = tf.train.Saver(var_list=G_vars)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path)
    summary_writer.add_graph(graph=sess.graph)        
    gstep=1
    for e in range(epochs):
        g_loss = []
        d_loss = [] 
        for ii in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2 - 1
            batch_z = np.random.uniform(-1, 1, size=(batch_size, 100))
            _,dl,summary1 = sess.run([D_optim,dcost,d_summary] ,feed_dict={x: batch_images, z_dim: batch_z})
            _ ,gl,summary2= sess.run([G_optim,gcost,g_summary], feed_dict={z_dim: batch_z})
            summary_writer.add_summary(summary1,gstep)
            summary_writer.add_summary(summary2,gstep)
            gstep+=1
            d_loss.append([dl])
            g_loss.append([gl])
        fixed_p = root + 'Fixed_results/' + model + str(e + 1) + '.png'
        show_result((e + 1), save=True, path=fixed_p)
        print('Epoch', e, 'completed out of',epochs,'generator loss:',np.mean(g_loss),'discriminator loss:',np.mean(d_loss))
        saver.save(sess, './checkpoints/generator.ckpt')
    images = []
    for e in range(epochs):
        img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
        images.append(imageio.imread(img_name))
    imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)

# The Generated Samples 
<img src="images/MNIST_GAN_generation_animation.gif">

In [14]:
print("Run the command line:\n" \
          "--> tensorboard --logdir=vgan " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Run the command line:
--> tensorboard --logdir=vgan 
Then open http://0.0.0.0:6006/ into your web browser


# The Graph Of the Network
<img src="images/graph.png">

## The Discriminator and Generator Losses
<img src="images/lossgd.png">